# DSBDL Assignment 02 - Data Wrangling 2

Create an “Academic performance” dataset of students and perform the following operations 
using Python. 
1. Scan all variables for missing values and inconsistencies. If there are missing values 
and/or inconsistencies, use any of the suitable techniques to deal with them.
2. Scan all numeric variables for outliers. If there are outliers, use any of the suitable 
techniques to deal with them.
3. Apply data transformations on at least one of the variables. The purpose of this 
transformation should be one of the following reasons: to change the scale for better 
understanding of the variable, to convert a non-linear relation into a linear one, or to 
decrease the skewness and convert the distribution into a normal distribution.
Reason and document your approach properly.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

## Create a dummy dataset

In [8]:
import random
import string

subjects = [ "AI" , "DSBDA" , "CC" , "WT" ]
missing_prob = 0.1
num_samples = 100

# Reference -> https://stackoverflow.com/questions/2257441/random-string-generation-with-upper-case-letters-and-digits
def get_student_name() -> str:
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

def get_random_marks() -> int:
    return random.randint( 0 , 50 )

def get_division() -> str:
    return "TE" + str( random.randint( 1 , 4 ) )

def is_nan() -> bool:
    return random.uniform( 0 , 1 ) < missing_prob

student_ds = {}
student_ds[ "name" ] = [ get_student_name() for _ in range( num_samples ) ]
student_ds[ "division" ] = [ get_division() for _ in range( num_samples ) ]
for subject in subjects:
    student_ds[ subject ] = [ get_random_marks() for _ in range( num_samples ) ]
    for i in range( num_samples ):
        if is_nan():
            student_ds[ subject ][ i ] = None

ds = pd.DataFrame.from_dict( student_ds )
ds.head()

,name,division,AI,DSBDA,CC,WT
0,L14OO,TE1,21.0,2.0,16.0,8.0
1,1110W,TE4,1.0,8.0,31.0,31.0
2,XBQPB,TE4,38.0,0.0,10.0,3.0
3,KAF3S,TE2,29.0,38.0,20.0,29.0
4,AX93S,TE3,23.0,20.0,NaN,45.0
